In [1]:
import sys
from traceback import format_exc
sys.path.append("../")
from trace_decorator import trace

In [2]:
s = "abcdefg"
ix = s.find("bc")
s[ix:]

'bcdefg'

In [5]:
global_var = "Simon Says"

@trace
def a_function():

    i = 0
    i += 1

    s = "a"
    s += "b"
    s += "c"

    l = []
    l.append(i)
    l.append(s)
    l.append(l)

    t = global_var
#     t = 999

    print(s + s + s)
    for i in range(4):
        i_sq_a_a_a_a = i ** 2
    
    return t

a_function()

    @trace
    def a_function():                                 | 
                                                      | 
        i = 0                                         | i = 0 
        i += 1                                        | i = 1 
                                                      | 
        s = "a"                                       | s = 'a' 
        s += "b"                                      | s = 'ab' 
        s += "c"                                      | s = 'abc' 
                                                      | 
        l = []                                        | l = [] 
        l.append(i)                                   | l = [1] 
        l.append(s)                                   | l = [1, 'abc'] 
        l.append(l)                                   | l = [1, 'abc', [...]] l = [1, 'abc', [...]] 
                                                      | 
        t = global_var                                | t = 'Simon Says' 
    

'Simon Says'

In [6]:
len("*** Trace Output *** " + "-" * 70), len("*** END Trace Output ***" + "-" * 67)

(91, 91)